In [1]:
import math
import tensorflow as tf
import numpy as np
from mingpt.model import GPT, GPTConfig

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
class AdditionDataset():
    """
    Returns addition problems of up to some number of digits in the inputs. Recall
    that all GPT cares about are sequences of integers, and completing them according to
    patterns in the data. Therefore, we have to somehow encode addition problems
    as a sequence of integers.
    
    The sum of two n-digit numbers gives a third up to (n+1)-digit number. So our
    encoding will simply be the n-digit first number, n-digit second number, 
    and (n+1)-digit result, all simply concatenated together. Because each addition
    problem is so structured, there is no need to bother the model with encoding
    +, =, or other tokens. Each possible sequence has the same length, and simply
    contains the raw digits of the addition problem.
    
    As a few examples, the 2-digit problems:
    - 85 + 50 = 135 becomes the sequence [8, 5, 5, 0, 1, 3, 5]
    - 6 + 39 = 45 becomes the sequence [0, 6, 3, 9, 0, 4, 5]
    etc.
    
    We will also only train GPT on the final (n+1)-digits because the first
    two n-digits are always assumed to be given. So when we give GPT an exam later,
    we will e.g. feed it the sequence [0, 6, 3, 9], which encodes that we'd like
    to add 6 + 39, and hope that the model completes the integer sequence with [0, 4, 5]
    in 3 sequential steps.
    
    fun exercise: does it help if the result is asked to be produced in reverse order?
    """

    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1
        
        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __iter__(self):
        # given a problem index idx, first recover the associated a + b
        for idx in range(self.__len__()):
            idx = self.ixes[idx]
            nd = 10**self.ndigit
            a = idx // nd
            b = idx %  nd
            c = a + b
            render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028" 
            dix = [int(s) for s in render] # convert each character to its token index
            # x will be input to GPT and y will be the associated expected outputs
            x = dix[:-1]
            y = dix[1:] # predict the next token in the sequence
            y[:self.ndigit*2-1] = [-1] * (self.ndigit*2-1) # we will only train in the output locations. -100 will mask loss to zero
            x = tf.convert_to_tensor(x,dtype=tf.int32)
            y = tf.convert_to_tensor(y,dtype=tf.int32)
            yield x, y
            
    __call__ = __iter__

In [4]:
# create a dataset for e.g. 2-digit addition
ndigit = 2
train_dataset_gen = AdditionDataset(ndigit=ndigit, split='train')
test_dataset_gen = AdditionDataset(ndigit=ndigit, split='test')

In [5]:
train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))
test_dataset = tf.data.Dataset.from_generator(test_dataset_gen,(tf.int32,tf.int32))

In [6]:
# initialize a baby GPT model
mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size, 
                  n_layer=2, n_head=4, n_embd=128)
# model = GPT(mconf)

In [7]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=50, batch_size=512, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(train_dataset_gen)*(ndigit+1),
                      num_workers=4)
trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), test_dataset, len(test_dataset_gen), tconf)

/Users/raj/.virtualenvs/mingpt/lib/python3.7/site-packages/fastprogress/fastprogress.py:102: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [8]:
trainer.train()

epoch 1: train loss 5.81625. lr 5.994512e-04
epoch 1: test loss 5.05172.
epoch 2: train loss 4.87166. lr 5.977197e-04
epoch 2: test loss 4.57972.
epoch 3: train loss 4.41712. lr 5.948115e-04
epoch 3: test loss 4.02392.
epoch 4: train loss 4.06186. lr 5.907379e-04
epoch 4: test loss 3.77200.
epoch 5: train loss 3.78744. lr 5.855153e-04
epoch 5: test loss 3.50269.
epoch 6: train loss 3.60194. lr 5.791641e-04
epoch 6: test loss 3.32805.
epoch 7: train loss 3.45809. lr 5.717095e-04
epoch 7: test loss 3.20248.
epoch 8: train loss 3.37508. lr 5.631810e-04
epoch 8: test loss 3.21148.
epoch 9: train loss 3.30742. lr 5.536122e-04
epoch 9: test loss 3.06461.
epoch 10: train loss 3.21280. lr 5.430411e-04
epoch 10: test loss 2.94524.
epoch 11: train loss 3.14949. lr 5.315093e-04
epoch 11: test loss 2.85418.
epoch 12: train loss 3.06388. lr 5.190625e-04
epoch 12: test loss 2.81797.
epoch 13: train loss 2.96562. lr 5.057497e-04
epoch 13: test loss 2.54328.
epoch 14: train loss 2.75026. lr 4.916238e-

In [9]:
from mingpt.utils import sample

In [10]:
def give_exam(dataset, batch_size=32, max_batches=-1):
    
    results = []
    
    loader = dataset.batch(batch_size)
    for b, (x, y) in enumerate(loader):
        d1d2 = x[:, :ndigit*2]
        d1d2d3 = sample(trainer.model, d1d2, ndigit+1)
        d3 = d1d2d3[:, -(ndigit+1):]
        factors = tf.convert_to_tensor([[10**i for i in range(ndigit+1)][::-1]])
        # decode the integers from individual digits
        d1i = tf.reduce_sum((d1d2[:,:ndigit] * factors[:,1:]),axis=1)
        d2i = tf.reduce_sum((d1d2[:,ndigit:ndigit*2] * factors[:,1:]),axis=1)
        d3i_pred = tf.reduce_sum((d3 * factors),axis=1)
        d3i_gt = d1i + d2i
        correct = (d3i_pred == d3i_gt) # Software 1.0 vs. Software 2.0 fight RIGHT on this line, lol
        for i in range(x.shape[0]):
            results.append(int(correct[i]))
            judge = 'YEP!!!' if correct[i] else 'NOPE'
            if not correct[i]:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)" 
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))
        
        if max_batches >= 0 and b+1 >= max_batches:
            break

    print("final score: %d/%d = %.2f%% correct" % (np.sum(results), len(results), 100*np.mean(results)))

In [11]:
give_exam(train_dataset,batch_size=32,max_batches=-1)

GPT claims that 067 + 052 = 129 (gt is 119; NOPE)
GPT claims that 069 + 050 = 129 (gt is 119; NOPE)
GPT claims that 068 + 051 = 129 (gt is 119; NOPE)
GPT claims that 033 + 037 = 060 (gt is 070; NOPE)
GPT claims that 059 + 050 = 119 (gt is 109; NOPE)
GPT claims that 031 + 039 = 060 (gt is 070; NOPE)
GPT claims that 034 + 036 = 060 (gt is 070; NOPE)
GPT claims that 032 + 038 = 060 (gt is 070; NOPE)
GPT claims that 058 + 051 = 119 (gt is 109; NOPE)
GPT claims that 035 + 035 = 060 (gt is 070; NOPE)
GPT claims that 095 + 095 = 180 (gt is 190; NOPE)
GPT claims that 058 + 050 = 118 (gt is 108; NOPE)
final score: 8988/9000 = 99.87% correct


In [12]:
give_exam(test_dataset,batch_size=32,max_batches=-1)

GPT claims that 091 + 099 = 180 (gt is 190; NOPE)
final score: 999/1000 = 99.90% correct
